In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [68]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the base URL
base_url = 'https://www.makaan.com/listings?propertyType=apartment,villa,independent-house&newOrResale=new&furnished=unfurnished&listingType=buy&pageType=LISTINGS_PROPERTY_URLS&cityName=Hyderabad&cityId=12&templateId=MAKAAN_CITY_LISTING_BUY'
#The above url has the property details of Unfurnished- Apartment/Villa/Independent Houses

# Define the list to store locality data
locality_extracted = []
price_per_sft_extracted=[]
area_sft_extracted=[]
property_type_extracted = []


# Loop through the specified page numbers
for page_number in range(1,850):
    # Construct the URL for the current page
    url = f'{base_url}&page={page_number}'
    
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all required elements 
        locality_on_current_page = soup.find_all('span', itemprop='addressLocality')
        price_per_sft_on_current_page=soup.find_all('td', class_='lbl rate')
        area_sft_on_current_page=soup.find_all('td', class_='size')
        
        # Extend the extracted lists with the data from the current page
        locality_extracted.extend(locality_on_current_page)
        price_per_sft_extracted.extend(price_per_sft_on_current_page)
        area_sft_extracted.extend(area_sft_on_current_page)
        
         # Find all <a> elements with the specified class and itemprop='name' to extract property type
        property_type_elements = soup.find_all('a', class_='typelink', itemprop='url')
        
        # Extract the property type text and append it to the property_type_extracted list
        property_type_extracted.extend([element.find('meta', itemprop='name')['content'] for element in property_type_elements])
        
    else:
        print(f'Failed to retrieve data for page {page_number}. Status code:', response.status_code)

# Create a DataFrame from the extracted locality data
df = pd.DataFrame({'Locality': [i.text for i in locality_extracted],
                   'Price_per_sft': [k.text for k in price_per_sft_extracted],
                   'Area' : [j.text for j in area_sft_extracted ],
                   'Property_Type': property_type_extracted})

# Print the DataFrame
print(df)


                Locality  Price_per_sft   Area             Property_Type
0      Gundlapochampally  4,200 / sq ft  1571   3 BHK Apartment for sale
1               Ameenpur  5,400 / sq ft  1065   2 BHK Apartment for sale
2             Patancheru  7,700 / sq ft  1930       4 BHK Villa for sale
3                Miyapur  6,500 / sq ft  1967   3 BHK Apartment for sale
4            Uppal Kalan  6,104 / sq ft  1065   2 BHK Apartment for sale
...                  ...            ...    ...                       ...
16975           Ameenpur  4,800 / sq ft  1600   3 BHK Apartment for sale
16976           Ameenpur  4,800 / sq ft  1150   2 BHK Apartment for sale
16977           Ameenpur  4,800 / sq ft  1150   2 BHK Apartment for sale
16978           Ameenpur  4,800 / sq ft  1600   3 BHK Apartment for sale
16979           Ameenpur  4,800 / sq ft  1600   3 BHK Apartment for sale

[16980 rows x 4 columns]


In [69]:
df.to_csv('Hyd_real_estate.csv')

In [3]:
df2=pd.read_csv('Hyd_real_estate.csv')

In [4]:
df2.head()

,Unnamed: 0,Locality,Price_per_sft,Area,Property_Type
0,0,Gundlapochampally,"4,200 / sq ft",1571,3 BHK Apartment for sale
1,1,Ameenpur,"5,400 / sq ft",1065,2 BHK Apartment for sale
2,2,Patancheru,"7,700 / sq ft",1930,4 BHK Villa for sale
3,3,Miyapur,"6,500 / sq ft",1967,3 BHK Apartment for sale
4,4,Uppal Kalan,"6,104 / sq ft",1065,2 BHK Apartment for sale


In [5]:
df2['Price_per_sft'].describe()

count             16980
unique             2953
top       4,999 / sq ft
freq                551
Name: Price_per_sft, dtype: object

In [6]:
df2['Price_per_sft'] = df2['Price_per_sft'].str.replace(r'[^0-9]','',regex=True)

In [7]:
df2.head()

,Unnamed: 0,Locality,Price_per_sft,Area,Property_Type
0,0,Gundlapochampally,4200,1571,3 BHK Apartment for sale
1,1,Ameenpur,5400,1065,2 BHK Apartment for sale
2,2,Patancheru,7700,1930,4 BHK Villa for sale
3,3,Miyapur,6500,1967,3 BHK Apartment for sale
4,4,Uppal Kalan,6104,1065,2 BHK Apartment for sale


In [8]:
df2['Price_per_sft']=pd.to_numeric(df2['Price_per_sft'])

In [9]:
df2['Property_Type'].describe()

count                        16980
unique                          24
top       3 BHK Apartment for sale
freq                          7021
Name: Property_Type, dtype: object

In [10]:
df2['Property_Type'] = df2['Property_Type'].str.replace(' for sale','',regex=True)

In [11]:
df2['BHK number']=df2['Property_Type'].str.replace(r'[^0-9]','',regex=True)


In [12]:
df2.head()

,Unnamed: 0,Locality,Price_per_sft,Area,Property_Type,BHK number
0,0,Gundlapochampally,4200,1571,3 BHK Apartment,3
1,1,Ameenpur,5400,1065,2 BHK Apartment,2
2,2,Patancheru,7700,1930,4 BHK Villa,4
3,3,Miyapur,6500,1967,3 BHK Apartment,3
4,4,Uppal Kalan,6104,1065,2 BHK Apartment,2


In [13]:
df2['Property_Type'].unique()

array(['3 BHK Apartment', '2 BHK Apartment', '4 BHK Villa',
       '4 BHK Apartment', '4 BHK Independent House', '1 BHK Apartment',
       '5 BHK Apartment', '3 BHK Independent House', '2 BHK Villa',
       '3 BHK Villa', '5 BHK Villa', '2 BHK Independent House',
       '1 RK Studio Apartment', '5 BHK Independent House',
       '7 BHK Independent House', '10 BHK Independent House',
       '8 BHK Independent House', '1 BHK Independent House',
       '6 BHK Villa', '6 BHK Independent House',
       '9 BHK Independent House', '6 BHK Apartment', '1 BHK Villa',
       '10 BHK Apartment'], dtype=object)

In [14]:
df2['Property_Type']=df2['Property_Type'].str.replace('BHK','',regex=True)

In [15]:
df2['Property_Type'].unique()

array(['3  Apartment', '2  Apartment', '4  Villa', '4  Apartment',
       '4  Independent House', '1  Apartment', '5  Apartment',
       '3  Independent House', '2  Villa', '3  Villa', '5  Villa',
       '2  Independent House', '1 RK Studio Apartment',
       '5  Independent House', '7  Independent House',
       '10  Independent House', '8  Independent House',
       '1  Independent House', '6  Villa', '6  Independent House',
       '9  Independent House', '6  Apartment', '1  Villa',
       '10  Apartment'], dtype=object)

In [16]:
df2['Property_Type']=df2['Property_Type'].str.replace(r'\d+','',regex=True)

In [17]:
df2['Property_Type'].unique()

array(['  Apartment', '  Villa', '  Independent House',
       ' RK Studio Apartment'], dtype=object)

In [18]:
df2.columns

Index(['Unnamed: 0', 'Locality', 'Price_per_sft', 'Area', 'Property_Type',
       'BHK number'],
      dtype='object')

In [19]:
df2=df2.drop('Unnamed: 0',axis=1)

In [20]:
df2.columns

Index(['Locality', 'Price_per_sft', 'Area', 'Property_Type', 'BHK number'], dtype='object')

In [21]:
df2.to_csv('HyderabadRealEstate_cleaned.csv')

In [22]:
df2.head()

,Locality,Price_per_sft,Area,Property_Type,BHK number
0,Gundlapochampally,4200,1571,Apartment,3
1,Ameenpur,5400,1065,Apartment,2
2,Patancheru,7700,1930,Villa,4
3,Miyapur,6500,1967,Apartment,3
4,Uppal Kalan,6104,1065,Apartment,2


SyntaxError: invalid syntax (1118962691.py, line 1)